In [10]:
from datasets import load_dataset
import os
from tqdm import tqdm
import random

random.seed(42)

In [11]:
dataset_root_dir = "/data_ssd/huggingface_dataset"
cache_dir = "/data_ssd/huggingface_cache"

dataset_id = os.path.join(dataset_root_dir,"BAAI/DenseFusion-1M")

In [12]:
dataset = load_dataset(dataset_id, cache_dir=cache_dir,name="DenseFusion-1M",split="train") # ['DenseFusion-4V-100K', 'DenseFusion-1M']

In [22]:
print(dataset[0].keys())
print("Number of samples:", len(dataset))
print("First sample:", dataset[1])

dict_keys(['image_id', 'caption', 'url', 'image_path'])
Number of samples: 1058790
First sample: {'image_id': '3853221003038', 'caption': 'The image is a black and white photograph featuring a collection of firearm parts laid out in rows. The parts are metallic with various textures, including cylindrical shapes, bolts, and other mechanical components. The focus is on the central part of the image, where the parts are densely packed, and the details of each part are less discernible due to the depth of field.\n\nOverlaying the photograph is a circular logo with a red border and a black background. The logo contains white text that reads "Registered International Traffic in Arms Registrations" and the acronym "ITAR" in large, bold red letters. The logo is centered and occupies a significant portion of the image, partially obscuring the view of the firearm parts behind it.\n\nThe layout of the image is such that the logo and the parts occupy the entire frame, with no discernible backgrou

In [ ]:
prompt_list_image_textualization = ['<image>\nWhat do you think is going on in this snapshot?', '<image>\nCan you describe the main features of this image for me?', '<image>\nCan you elaborate on the elements of the picture provided?', '<image>\nWhat are the key elements in this picture?', '<image>\nDescribe the following image.', "<image>\nWhat's happening in the scene?", '<image>\nWhat is this photo about?', '<image>\nAnalyze the image in a comprehensive and detailed manner.', '<image>\nWrite a detailed description of the given image.', '<image>\nExplain the visual content of the image in great detail.', '<image>\nWhat do you see happening in this image?']
prompt_list_sharegpt4o = ['<image>\nPlease describe in depth what this image is about.', '<image>\nCan you explain in detail what is happening in the picture?', '<image>\nPlease explain in detail the scene depicted in the picture.', '<image>\nPlease describe in detail the dynamic and static elements in this image.', '<image>\nPlease describe what is happening in the image.', '<image>\nCan you describe all the objects and characters in the picture?', '<image>\nDescribe this image.', '<image>\nPlease conduct an in-depth analysis of the scene in the picture.', '<image>\nPlease elaborate on what this image shows.', '<image>\nDescribe everything in the image', '<image>\nProvide a detailed description of the presented image.', '<image>\nWhat scene is mainly depicted in the picture?', '<image>\nDescribe the image, paying attention to its inner details.', '<image>\nWhat items or people are prominent in the picture?', '<image>\nDescribe the content of a given image in detail', '<image>\nPlease describe specifically what you observed in the picture and the possible scenes they might form.', '<image>\nWhat objects and people are shown in the picture?', '<image>\nPlease use detailed words to describe what the picture is about.', '<image>\nPlease describe specifically what this image is about.', '<image>\nPlease explain the scene depicted in the picture.', '<image>\nWhat scene is this picture depicting?', '<image>\nDescribe every detail in the picture.', '<image>\nWhat objects and people are in the picture?', '<image>\nWhat are the striking details of this image?', '<image>\nWhat is compelling about this image?', '<image>\nProvide a detailed description of the image.', '<image>\nProvide a detailed description of the main elements in the image.', '<image>\nPlease describe the scene depicted in this picture.', '<image>\nCan you describe what this image is about?', '<image>\nDescribe the given image', '<image>\nWhat elements and people are included in the image?', '<image>\nPlease explain the visual content of the image in detail.', '<image>\nPlease describe the content of the picture.', '<image>\nCan you explain what this image shows?', '<image>\nCan you analyze and elaborate on all the elements and details shown in this image?', '<image>\nWrite a detailed and comprehensive description of the image.', '<image>\nCan you list all the objects and people in the picture?', '<image>\nDescribe what the picture shows?', '<image>\nDescribe all the elements in the picture.', '<image>\nCan you describe the main elements in the image?', '<image>\nPlease interpret and describe each detail of this image and the overall scene they create.', '<image>\nPlease describe the situation in the picture as clearly as possible.', '<image>\nPlease describe the event or scene this image shows.', '<image>\nAnalyze and describe in detail the visual elements in this image.', '<image>\nCan you describe this photo in detail?', '<image>\nPlease describe the scene in the picture.', '<image>\nWhat details are worth noting in the picture?']
prompt_list = list(set(prompt_list_image_textualization + prompt_list_sharegpt4o))
prompt_list = \
[
 '<image>\nExplain the visual content of the image in great detail.', 
 '<image>\nPlease explain the visual content of the image in detail.', 
 
 '<image>\nPlease describe in depth what this image is about.',
 '<image>\nCan you explain in detail what is happening in the picture?',
 '<image>\nWrite a detailed description of the given image.', 
 '<image>\nPlease elaborate on what this image shows.',
 '<image>\nWhat items or people are prominent in the picture?',
 '<image>\nCan you elaborate on the elements of the picture provided?',
 '<image>\nAnalyze the image in a comprehensive and detailed manner.',
  '<image>\nWhat are the striking details of this image?', 
  '<image>\nWrite a detailed and comprehensive description of the image.',
  
  '<image>\nDescribe the image, paying attention to its inner details.', 
  '<image>\nAnalyze and describe in detail the visual elements in this image.', 
  '<image>\nDescribe the content of a given image in detail', 

  '<image>\nDescribe every detail in the picture.', 
  '<image>\nPlease explain in detail the scene depicted in the picture.', 
  '<image>\nPlease conduct an in-depth analysis of the scene in the picture.',
  '<image>\nCan you describe all the objects and characters in the picture?',
  '<image>\nProvide a detailed description of the presented image.', 
  '<image>\nPlease use detailed words to describe what the picture is about.',

   '<image>\nDescribe everything in the image',
   '<image>\nPlease describe specifically what you observed in the picture and the possible scenes they might form.',

   '<image>\nPlease interpret and describe each detail of this image and the overall scene they create.',
   '<image>\nDescribe all the elements in the picture.',
    '<image>\nProvide a detailed description of the image.',
    '<image>\nCan you describe this photo in detail?',
    '<image>\nCan you analyze and elaborate on all the elements and details shown in this image?',]
def make_conversation(id,image_path,question,answer,image_folder_root=None):
    if image_folder_root is not None:
        image_path = os.path.join(image_folder_root, image_path)
    return_data =   {
        "id": id,
        "image": image_path,
        "conversations": [
        {
            "from": "human",
            "value": f"{question}"
        },
        {
            "from": "gpt",
            "value": answer
        },
        ],
    }
    return return_data


In [20]:
for prompt in prompt_list:
    print(prompt)

<image>
Please explain the scene depicted in the picture.
<image>
Describe this image.
<image>
Provide a detailed description of the main elements in the image.
<image>
Describe the given image
<image>
Explain the visual content of the image in great detail.
<image>
Please explain the visual content of the image in detail.
<image>
Describe the following image.
<image>
Can you describe the main features of this image for me?
<image>
Please describe in depth what this image is about.
<image>
Can you explain in detail what is happening in the picture?
<image>
Write a detailed description of the given image.
<image>
Please elaborate on what this image shows.
<image>
What items or people are prominent in the picture?
<image>
Please describe the scene in the picture.
<image>
Can you elaborate on the elements of the picture provided?
<image>
Analyze the image in a comprehensive and detailed manner.
<image>
Please describe what is happening in the image.
<image>
Can you list all the objects an

In [21]:
print(prompt_list)

['<image>\nPlease explain the scene depicted in the picture.', '<image>\nDescribe this image.', '<image>\nProvide a detailed description of the main elements in the image.', '<image>\nDescribe the given image', '<image>\nExplain the visual content of the image in great detail.', '<image>\nPlease explain the visual content of the image in detail.', '<image>\nDescribe the following image.', '<image>\nCan you describe the main features of this image for me?', '<image>\nPlease describe in depth what this image is about.', '<image>\nCan you explain in detail what is happening in the picture?', '<image>\nWrite a detailed description of the given image.', '<image>\nPlease elaborate on what this image shows.', '<image>\nWhat items or people are prominent in the picture?', '<image>\nPlease describe the scene in the picture.', '<image>\nCan you elaborate on the elements of the picture provided?', '<image>\nAnalyze the image in a comprehensive and detailed manner.', '<image>\nPlease describe what

In [16]:
save_json_data = []

for i, sample in tqdm(enumerate(dataset), total=len(dataset)):
    image_path = sample["image_path"]
    question = random.choice(prompt_list)
    answer = sample["caption"]
    id = f"DenseFusion-1M-{sample["image_id"]}"
    
    conversation = make_conversation(id=id, image_path=image_path, question=question, answer=answer)
    
    save_json_data.append(conversation)
    break

  0%|          | 0/1058790 [00:00<?, ?it/s]


In [17]:
print(save_json_data)

[{'id': 'DenseFusion-1M-3340022002763', 'image': 'DenseFusion-1M/000000/3340022002763.png', 'conversations': [{'from': 'human', 'value': '<image>\nCan you describe the main features of this image for me?'}, {'from': 'gpt', 'value': 'The image displays four different styles of foldable hard cases designed for sunglasses, each with a unique color and pattern. At the top of the image, there is a logo with the text "MF SUNNIES" and the hashtag "#SE01" below it. The cases are arranged in a 2x2 grid against a white background.\n\nIn the top left, there is a beige case with a smiley face design featuring various expressions such as winking, smiling, and sleepy. The top right case is purple with a pattern of black fish faces and bubbles.\n\nThe bottom left case is light blue with a design of black sunglasses and the word "smile" in different fonts and orientations. The bottom right case is dark brown with a pattern of cartoonish fish faces and bubbles, similar to the top right case but with a 

In [10]:
import json
import tqdm

def load_json(file_path):
    """
    Load a JSON file and return its content as a Python dictionary.

    Parameters:
        file_path (str): The path to the JSON file.

    Returns:
        dict: The content of the JSON file as a dictionary.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data
def save_json(data, file_path):
    """
    Save a Python dictionary to a JSON file.

    Parameters:
        data (dict): The data to save.
        file_path (str): The path where the JSON file will be saved.
    """
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

In [11]:
data = load_json("/data_ssd/DenseFusion1M/densefusion1m-train.json")

In [12]:
image_root_dir = "/data_ssd/huggingface_dataset/BAAI/DenseFusion-1M/images"

In [ ]:
import os

new_json_data = []
for item in tqdm.tqdm(data):
    image_path = item["image"]
    image_path = os.path.join(image_root_dir, image_path)
    if not os.path.exists(image_path):
        print(f"Image path does not exist: {image_path}")
    else:
        new_json_data.append(item)

  6%|▌         | 58693/1058790 [00:12<03:55, 4237.78it/s]

Updated image path: /data_ssd/huggingface_dataset/BAAI/DenseFusion-1M/images/DenseFusion-1M/000072/1278740002463.png


  9%|▉         | 96761/1058790 [00:21<03:25, 4678.88it/s]

Updated image path: /data_ssd/huggingface_dataset/BAAI/DenseFusion-1M/images/DenseFusion-1M/000099/814546033157.png


 10%|█         | 106598/1058790 [00:23<03:46, 4212.07it/s]

...

 94%|█████████▍| 993608/1058790 [03:34<00:14, 4403.64it/s]

Updated image path: /data_ssd/huggingface_dataset/BAAI/DenseFusion-1M/images/DenseFusion-1M/000099/1084772004006.png


 99%|█████████▉| 1051398/1058790 [03:46<00:01, 4838.20it/s]

Updated image path: /data_ssd/huggingface_dataset/BAAI/DenseFusion-1M/images/DenseFusion-1M/000072/3597269006784.png


100%|██████████| 1058790/1058790 [03:48<00:00, 4633.75it/s]


In [14]:
save_json(new_json_data, "/data_ssd/DenseFusion1M/densefusion1m-train-existed.json")

In [15]:
print(len(new_json_data))

1058755
